<p class="float-right">![logo](https://github.com/mmattamala/LogosFCFM/blob/master/Ciencias%20de%20la%20Computaci%C3%B3n/Logos%20Facultad/fcfm_dcc_png.png?raw=true =300x)</p>

# Informe Minería de Datos

CC5206 - Introducción a la Mineria de Datos 

Profesora: Bárbara Poblete

Alumnos: \
Cristián Tamblay \
José Cañete \
Diego Díaz
                  

Auxiliares: \
Hernán Sarmiento \
José Miguel Herrera

#Hito 1
###Motivación: 

¿Cuál es el contexto general del tema/problema/datos que eligió
estudiar?

- El contexto general son las competencias de Programación Competitiva, éstas son competencias en las que programadores deben pensar y resolver un set de problemas de manera correcta (es decir, eficaces y con soluciones óptimas dado el tamaño del input) y en el menor tiempo posible. En ellas, como el nombre lo dice, se compite entre equipos, quedando en mejor lugar quien tiene más problemas resueltos y en caso de haber varios equipos con la misma cantidad de problemas resueltos, el que haya sido más rápido programando sus soluciones, es decir, que su tiempo acumulado sea menor.
Ahora bien, se trabajará con problemas de estas competencias con el fin de caracterizarlos y clasificarlos. Se intentará separar los problemas en categorías de temas, en particular en cómo se resuelven (por ejemplo: programación dinámica y búsqueda binaria) y eventualmente por dificultad del problema. Lo que se intentará es crear un clasificador de estos problemas y en lo posible un recomendador de ellos. Los datos que se eligieron para esta ocasión corresponden a un dataset de problemas de Codeforces.


¿Por qué podría ser de interés estos datos?

- En primer lugar porque las competencias de Programación Competitiva son muy populares en muchas partes del mundo en contextos en los que se estudia programación, ciencias de la computación y matemáticas. Tanto es así que existen varias competencias a nivel mundial de esto, en particular la IOI a nivel escolar y la ACM ICPC a nivel universitario. Estas competencias tienen un nivel de dificultad alto y para alcanzarlo, estudiantes de todo el mundo se preparan y entrenan constantemente resolviendo problemas. Por otra parte, es conocido que las grandes empresas tech como Google, Microsoft y Facebook entrevistan a sus aspirantes mediante la resolución de problemas de este tipo. Dado ese contexto, el crear un clasificador para de alguna manera facilitar este tipo de entrenamientos puede ser muy util.
Por otra parte, es interesante ver si hay alguna relación entre el texto que explica el problema, y el problema computacional que se quiere resolver. Esto debido a que en general los problemas cuentan una historia, que puede ser muy variada en su temática. Además, se podría tratar de encontrar alguna relación con el nivel de dificultad de éste. 
Por último, también es interesante notar que si bien los problemas pueden categorizarse de manera individual en clases distintas, también pueden categorizarse en más de una al mismo tiempo. El ejemplo clásico de esto son aquellos problemas más avanzados y difíciles en los que las técnicas más básicas (como búsqueda binaria) se usan de manera constante y sin ser el foco principal del problema.

###Temática o problemática central:
¿Qué es lo que les gustaría analizar utilizando estos datos? o ¿Qué problema les gustaría resolver utilizando estos datos?

- Encontrar relaciones entre el texto que describe el problema (y que se cuenta como una historia en la mayoría de los casos) y el problema computacional a resolver, o al menos a qué tipo (tema) de problema corresponde. Para ello el foco estaría en analizar tres aspectos: el texto que describe el problema, el que describe el input y el que describe el output. También aprovecharíamos el hecho de que muchos problemas ya están clasificados. Todo esto sería la base para hacer el clasificador de temas y así, eventualmente poder usar la clasificación anterior en conjunto con las estadísticas de resolución de los problemas, para hacer un clasificador de dificultad y un recomendador de problemas. Una hipótesis inicial en cuanto a la clasificación de temas, es que existen palabras claves que logran describir ciertos temas (por ejemplo, si se menciona "buscar" "máximo" "tal que" "orden" es probable que el problema se resuelva con búsqueda binaria).

¿Cómo se se puede hacer esto de forma preliminar?

- De forma preliminar, lo que se espera es hacer análisis de texto para encontrar patrones, palabras claves, etc. Para ello lo primero planteado es hacer un modelo Bag of Words con los textos que describen el problema, el input y el output con el fin de obtener características y que éstas se puedan procesar de mejor manera. Además de esto, se podrían usar otras técnicas que permitan obtener más características, como lo son el uso de n-gramas o análisis de sentimiento.

###Descripción de los datos que se van a utilizar (análisis exploratorio):
 
 - Las principales características del dataset son: 
  - El texto en forma de historia que introduce el problema
  - Las especificaciones del input
  - Las especificaciones del output
  - El límite de memoria
  - El límite de tiempo
  - Los tags asociados

#### Cosas Útiles

Instalación de librerías necesarias para poder visualizar los gráficos que contienen características del dataset

In [34]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='cristian.tamblay', api_key='OJwr2lNHDmoskz1vzRTb')

def mostrar_todo(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(df)

####Importar Datos

Importar los datos adjuntos con el informe al ejecutar la siguiente ventana.

In [35]:
'''from google.colab import files
import os
files.upload()
!ls'''

'from google.colab import files\nimport os\nfiles.upload()\n!ls'

Verificar que output_all.csv esté en el entorno.

# Leer/importar datos

In [36]:
datos_problemas = pd.read_csv("output_all.csv")
datos_problemas.head()

#mostrar_todo(datos_problemas.main_text)
print(datos_problemas.shape)

(3788, 9)


# Arreglar formato de los datos

Lo primero es eliminar los problemas que no poseen descripción (main_text)

In [37]:
datos_problemas = datos_problemas[pd.notnull(datos_problemas['main_text'])]
print(datos_problemas.shape)

(3773, 9)


Luego arreglamos los tags

In [38]:
datos_problemas.tags = datos_problemas.tags.apply(lambda x: [str(elem.strip()).replace("'", "") for elem in x[1:len(x)-1].split('/')])

Lo siguiente que haremos es separar el dataset en dos: una parte que posee al menos un tag y otra que no posee tags.

In [39]:
datos_notageados = datos_problemas[datos_problemas.tags.apply(lambda x: (len(x) == 1 and '' in x))]
datos_tageados = datos_problemas[datos_problemas.tags.apply(lambda x: not(len(x) == 1 and '' in x))]

Ahora lo que haremos es encontrar el set de categorias y luego armaremos un dataframe separado por categorias

In [40]:
categorias = []
for fila in datos_tageados.tags:
    for cat in fila:
        categorias.append(cat)
categorias = list(set(categorias))

Ahora crearemos un nuevo dataframe con columnas de cada categoria que contienen un 0 o un 1 dependiendo de si ese texto pertenece a esa categoria

In [41]:
columns = ['main_text','tags'] + categorias
#print(columns)
datos_por_categoria = pd.DataFrame(columns=columns)
#datos_por_categoria.head()
datos_por_categoria.main_text = datos_tageados.main_text
datos_por_categoria.tags = datos_tageados.tags

#datos_por_categoria.head()

datos_por_categoria.fillna(0)

for columna in datos_por_categoria.columns:
    if (columna != 'main_text' and columna != 'tags'):
        for index, fila in datos_por_categoria.iterrows():
            if (columna in fila['tags']):
                fila[columna] = 1
            else:
                fila[columna] = 0

datos_por_categoria.drop('tags', axis=1, inplace=True)

# Analisis preliminar de los datos

Analizaremos ciertas características del dataset de problemas, partiendo por ver cúanto es el tiempo límite promedio en que deben resolverse.

In [42]:
cantidad = []
for i in range(0,len(datos_problemas.time_limit.unique())):
  cantidad.append(len(datos_problemas[datos_problemas.time_limit == datos_problemas['time_limit'].unique()[i]]))
cantidadPorTiempo = np.column_stack((np.asarray(cantidad),datos_problemas['time_limit'].unique()))
cantidadPorTiempo=cantidadPorTiempo[(-cantidadPorTiempo[:,0]).argsort()] #El menos es decreasing
trace0 = go.Bar(
            x=cantidadPorTiempo[:,1],
            y=cantidadPorTiempo[:,0]
    )
data=[trace0]
layout = go.Layout(
            title='Cantidad de problemas en función del tiempo máximo',
            xaxis=dict(
                type='category',
                title='Tiempo máximo para resolver el problema'
            ),
            yaxis=dict(
                title='Cantidad de problemas'
            )
)
fig=go.Figure(data=data, layout=layout)
py.iplot(fig, filename='tiempo')

Se observa que la mayor parte de este dataset se divide entre los problemas de 1 y 2 segundos, lo que no nos aporta mucho sobre la clasificación de los problemas. Pueden haber problemas de temas muy distintos con el mismo tiempo de resolución. \
Haciendo el mismo análisis, pero esta vez midiendo la cantidad de memoria que utilizan los problemas, se llega a lo siguiente.

In [43]:
cantidad = []
for i in range(0,len(datos_problemas.memory_limit.unique())):
  cantidad.append(len(datos_problemas[datos_problemas.memory_limit == datos_problemas['memory_limit'].unique()[i]]))
cantidadPorMemoria = np.column_stack((np.asarray(cantidad),datos_problemas['memory_limit'].unique()))
cantidadPorMemoria=cantidadPorMemoria[(-cantidadPorMemoria[:,0]).argsort()] #El menos es decreasing
trace0 = go.Bar(
            x=cantidadPorMemoria[:,1],
            y=cantidadPorMemoria[:,0]
    )
data=[trace0]
layout = go.Layout(
            title='Cantidad de problemas en función de la cantidad máxima de memoria disponible',
            xaxis=dict(
                type='category',
                title='Cantidad máxima de memoria para resolver el problema'
            ),
            yaxis=dict(
                title='Cantidad de problemas'
            )
)
fig=go.Figure(data=data, layout=layout)
py.iplot(fig, filename='memoria')

Se observa que casi todos los problemas tienen como límite los 256 MB de memoria. No se puede encontrar una rapida relación entre el tipo de problema y la cantidad de memoria entregada.

A continuación se observan los 30 tags más usados.

In [44]:
cantidad = []
posibles = []
for row in datos_problemas.tags:
    for tag in row:
        posibles.append(tag)
posibles = pd.Series(posibles).unique()
print(posibles)
print(len(posibles))
for i in range(0,len(posibles)):
  cantidad.append(len(datos_problemas[datos_problemas.tags.tolist() in posibles[i]]))
cantidadPorTag = np.column_stack((np.asarray(cantidad),datos_problemas['tags'].unique()))
cantidadPorTag=cantidadPorTag[(-cantidadPorTag[:,0]).argsort()] #El menos es decreasing
cantidadPorTag=cantidadPorTag[0:10,:]
trace0 = go.Bar(
            x=cantidadPorTag[:,1],
            y=cantidadPorTag[:,0]
    )
data=[trace0]
layout = go.Layout(
            title='Cantidad de problemas en función del tag',
            xaxis=dict(
                type='category',
                title='Tag del problema'
            ),
            yaxis=dict(
                title='Cantidad de problemas'
            )
)
fig=go.Figure(data=data, layout=layout)
py.iplot(fig, filename='tags')

['math' 'implementation' 'geometry' 'dp' 'number theory'
 'constructive algorithms' 'special problem' 'strings' 'data structures'
 'dfs and similar' 'greedy' 'binary search' 'probabilities' 'sortings'
 'trees' 'divide and conquer' 'brute force' 'dsu' 'graphs' 'flows'
 'graph matchings' 'chinese remainder theorem' 'ternary search'
 'combinatorics' 'bitmasks' 'matrices' '' 'hashing' 'expression parsing'
 'games' 'two pointers' 'string suffix structures' 'shortest paths'
 '2-sat' 'meet-in-the-middle' 'fft' 'schedules']
37


TypeError: 'in <string>' requires string as left operand, not list

Se ve que la mayoría de los problemas tiene como tag "implementation", lo cual no dice mucho del problema. Adicionalmente, la segunda categoría más popular de problema son aquellos que nisiquiera poseen un tag descriptor, quedando ambigua la naturaleza de éste. Finalmente se puede observar que existen categorías que se repiten, pues están listadas con categorías adicionales, lo cual puede provocar una duplicación de problemas en caso de querer individualizarlos.

# Exploración Preliminar del Texto

En primer lugar, dado que la mayoría de la información deseamos obtenerla del texto (principal, de input y de output) se decidió hacer uso de la librería SKLearn que entre sus variados métodos ofrece la posibilidad de obtener Bag of Words a partir de texto así como N-Gramas. En este caso particular se decidió probar el uso de Bag of Words de manera sencilla para ver que tal funciona. Como resultado, con el siguiente código se puede ver el Bag of Words creado, donde cada texto inicial se convierte en un vector y podemos ver el vocabulario con las frecuencias de cada palabra en el conjunto de datos.

# Preparación de datos para clasificación

In [45]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns

In [46]:
train, test = train_test_split(datos_por_categoria, random_state=42, test_size=0.33, shuffle=True)
X_train = train.main_text
X_test = test.main_text

# Clasificación con Naive Bayes

In [47]:
# Define a pipeline combining a text feature extractor with multi lable classifier
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in categorias:
    #print(category)
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    #print(train[category].shape)
    NB_pipeline.fit(X_train, np.asarray(train[category], dtype="|S6"))
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(np.asarray(test[category], dtype="|S6"), prediction)))

... Processing constructive algorithms
Test accuracy is 0.8960396039603961
... Processing fft
Test accuracy is 0.9966996699669967
... Processing chinese remainder theorem
Test accuracy is 0.9983498349834984
... Processing combinatorics
Test accuracy is 0.943069306930693
... Processing shortest paths
Test accuracy is 0.9801980198019802
... Processing geometry
Test accuracy is 0.9488448844884488
... Processing ternary search
Test accuracy is 0.9933993399339934
... Processing string suffix structures
Test accuracy is 0.9892739273927392
... Processing dsu
Test accuracy is 0.9711221122112211
... Processing games
Test accuracy is 0.9785478547854786
... Processing implementation
Test accuracy is 0.7046204620462047
... Processing bitmasks
Test accuracy is 0.9595709570957096
... Processing meet-in-the-middle
Test accuracy is 0.995049504950495
... Processing special problem
Test accuracy is 0.9694719471947195
... Processing schedules
Test accuracy is 0.9991749174917491
... Processing dp
Test acc

# Clasificación con SVC

In [48]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categorias:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, np.asarray(train[category], dtype="|S6"))
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(np.asarray(test[category], dtype="|S6"), prediction)))

... Processing constructive algorithms
Test accuracy is 0.8927392739273927
... Processing fft
Test accuracy is 0.9966996699669967
... Processing chinese remainder theorem
Test accuracy is 0.9983498349834984
... Processing combinatorics
Test accuracy is 0.943069306930693
... Processing shortest paths
Test accuracy is 0.9801980198019802
... Processing geometry
Test accuracy is 0.9537953795379538
... Processing ternary search
Test accuracy is 0.9933993399339934
... Processing string suffix structures
Test accuracy is 0.9892739273927392
... Processing dsu
Test accuracy is 0.971947194719472
... Processing games
Test accuracy is 0.9818481848184818
... Processing implementation
Test accuracy is 0.683993399339934
... Processing bitmasks
Test accuracy is 0.9587458745874587
... Processing meet-in-the-middle
Test accuracy is 0.995049504950495
... Processing special problem
Test accuracy is 0.9760726072607261
... Processing schedules
Test accuracy is 0.9991749174917491
... Processing dp
Test accur

# Clasificación con Regresión Logistica

In [49]:
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categorias:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, np.asarray(train[category], dtype="|S6"))
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(np.asarray(test[category], dtype="|S6"), prediction)))

... Processing constructive algorithms
Test accuracy is 0.8960396039603961
... Processing fft
Test accuracy is 0.9966996699669967
... Processing chinese remainder theorem
Test accuracy is 0.9983498349834984
... Processing combinatorics
Test accuracy is 0.943069306930693
... Processing shortest paths
Test accuracy is 0.9801980198019802
... Processing geometry
Test accuracy is 0.9496699669966997
... Processing ternary search
Test accuracy is 0.9933993399339934
... Processing string suffix structures
Test accuracy is 0.9892739273927392
... Processing dsu
Test accuracy is 0.9711221122112211
... Processing games
Test accuracy is 0.9785478547854786
... Processing implementation
Test accuracy is 0.7087458745874587
... Processing bitmasks
Test accuracy is 0.9595709570957096
... Processing meet-in-the-middle
Test accuracy is 0.995049504950495
... Processing special problem
Test accuracy is 0.9694719471947195
... Processing schedules
Test accuracy is 0.9991749174917491
... Processing dp
Test acc

# Otros códigos eventualmente utiles

In [ ]:
### COMPLETAR ESTE CÓDIGO

## run_classifier recibe un clasificador, un dataset (X, y) 
## y opcionalmente la cantidad de resultados que se quiere obtener del clasificador

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score


def run_classifier(clf, X, y, num_tests=100):
    metrics = {'f1-score': [], 'precision': [], 'recall': []}
    
    for _ in range(num_tests):
        
        ### INICIO COMPLETAR ACÁ 
        #### TIP: divida el dataset, entrene y genere las predicciones.
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, stratify=y)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
      
        ### FIN COMPLETAR ACÁ
        
        
        metrics['f1-score'].append(f1_score(y_test, predictions))  # X_test y y_test deben ser definidos previamente
        metrics['recall'].append(recall_score(y_test, predictions))
        metrics['precision'].append(precision_score(y_test, predictions))
    
    return metrics

In [ ]:
## ejecutar este código

from sklearn.datasets import load_breast_cancer
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC  # support vector machine classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier

bc = load_breast_cancer()    # dataset cancer de mamas
X = bc.data
y = bc.target

c0 = ("Base Dummy", DummyClassifier(strategy='stratified'))
c1 = ("Decision Tree", DecisionTreeClassifier())
c2 = ("Gaussian Naive Bayes", GaussianNB())
c3 = ("KNN", KNeighborsClassifier(n_neighbors=5))

classifiers = [c0, c1, c2, c3]

for name, clf in classifiers:
    metrics = run_classifier(clf, X, y)   # hay que implementarla en el bloque anterior.
    print("----------------")
    print("Resultados para clasificador: ",name) 
    print("Precision promedio:",np.array(metrics['precision']).mean())
    print("Recall promedio:",np.array(metrics['recall']).mean())
    print("F1-score promedio:",np.array(metrics['f1-score']).mean())
    print("----------------\n\n")